In [49]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu,SmoothingFunction
import pandas as pd
import numpy as np
import docker
import os
import re
import random
from string import punctuation
import json
import matplotlib.pyplot as plt
punc = punctuation
smooth = SmoothingFunction()

def mythril_get(bb):
    mythril_result = []
    myt = bb['mythril']
    line = bb['contract_line']
    for index,j in enumerate(myt):
        if j != None and type(j) != float and len(j) > 0:
            total = []
            for i in j:
                #print(i)
                ccode = i.get('code')
                formatted = {
                    #'address':i['address'],
                    #'code':ccode,
                    'function':i['function'],
                    'lineno':i['lineno'],
                    'title':i['title'],
                    'type':i['type']
                }
                
                if(line[index][0] <= formatted['lineno'] and line[index][1] >= formatted['lineno']):
                    total.append(formatted)
            mythril_result.append(total)
        else:
            mythril_result.append([])
    return mythril_result
def slither_get(bb):
    slither_result = []
    sli = bb['slither']
    line = bb['contract_line']
    for index,j in enumerate(sli):
        if j != None and type(j) != float and len(j) > 0:
            temp=[]
            for i in j:
                if len(i['elements']) > 0 and i['elements'][0].get('source_mapping'):
                    func = i['elements'][0].get('name')
                    t = {
                        'title':i['check'],
                        'function':func,
                        #'contract':i['elements'][0].get('contract').get('name'),
                        'lineno':i['elements'][0]['source_mapping']['lines'],
                        'type':i['impact']
                    }
                    #print("t:"+str(t['lineno'][0]))
                    #print('line'+str(line[index][0])+str(line[index][1]))
                    if(len(t['lineno']) > 0 and line[index][0] <= t['lineno'][0] and line[index][1] >= t['lineno'][0]):
                        temp.append(t)
            slither_result.append(temp)
        else:
            slither_result.append([])
    return slither_result
def osiris_get(bb):
    result = []
    osi = bb['osiris']
    line = bb['contract_line']
    count = 0
    for  index,i in enumerate(osi):
        if i != None and type(i) != float and len(i) > 0:
            temp = []  
            for j in i:
                error = j['errors']
                name = j['name']
                if len(error) > 0:
                    for ttt in error:
                        t = {
                            'title':ttt['message'],
                            'lineno':ttt['line'],
                            'function':name
                        }
                        if(line[index][0] <= t['lineno'] and line[index][1] >= t['lineno']):
                            temp.append(t)
            result.append(temp)
        else:
            result.append([])
    return result
def find_range(code,f):
    head = code.splitlines()[0]
    hang = f.splitlines()
    for i in range(len(hang)):
        if head in hang[i]:
            a = i
            flag = 1
            j = i+1
            while flag != 0 and j < len(hang):
                if '{' in hang[j]:
                    flag += hang[j].count('{')
                if '}' in hang[j]:
                    flag -= hang[j].count('}')
                j += 1
            return [a,j-1]
    return [0,0]
def find_func_range(code,f):
    head = code.splitlines()[0]
    hang = f.splitlines()
    for i in range(len(hang)):
        if head in hang[i]:
            a = i
            flag = 1
            j = i+1
            while flag != 0 and j < len(hang):
                if '{' in hang[j]:
                    flag += hang[j].count('{')
                if '}' in hang[j]:
                    flag -= hang[j].count('}')
                j += 1
            return [a,j-1]
    print("find_func_range error")
    return [0,0]
def get_contract_line(bb):
    code_range = []
    name = list(bb['code_name'])
    for i in range(len(name)):
        try:
            f = open('/home/yfliu/data_sol/'+name[i],encoding = 'utf-8').read()
        except:
            f = open('/home/yfliu/data_sol/'+name[i],encoding = 'GBK').read()
        code = bb.loc[i,'contract_code']
        con_range = find_range(code,f)
        code_range.append(con_range)
    return code_range
def get_function_line(bb):
    code_range = []
    name = list(bb['code_name'])
    for i in range(len(name)):
        try:
            f = open('/home/yfliu/data_sol/'+name[i],encoding = 'utf-8').read()
        except:
            f = open('/home/yfliu/data_sol/'+name[i],encoding = 'GBK').read()
        code = bb.loc[i,'contract_code']
        con_range = find_func_range(code,f)
        code_range.append(con_range)
    return code_range
def add_sol(sol_name,target):
    temp = target[sol_name]
    #print(temp[0])
    for i in range(len(temp)):
        if '.sol' not in temp[i]:
            temp[i] += '.sol'
    target[sol_name] = temp
def check_len(x):
    return len(x)
def print_code(addr):
    ttt = open("/home/yfliu/data_sol/"+addr).read().splitlines()
    for index,i in enumerate(ttt):
        print(str(index)+i)
def func_code(coindex):
    ana = bb[bb['code_index'] == coindex].iloc[0]
    #print(ana['contract_code'])
    for j in ['osiris_analysis','mythril_analysis_not_info','slither_analysis_not_info']:
        print(j)
        for i in ana[j]:
            print(i)
        print()
    def spli(x):
        return x.split('-')[0]
    def include_f(x,y):
        return y in eval(x)['href']
    temp2 = bb[bb['code_index'] == coindex]['sni_name'].apply(spli)
    print(list(temp2))
    for i in temp2:
        temp = post[post['info'].apply(include_f,y = i)]
        print(eval(temp.iloc[0]['info'])['href'])
    print_code(ana['code_name'])
def smallerRange(oldNum, lineNum):
    if len(oldNum) < 1 or len(lineNum) < 1:
        print('great error')
        return False
    oldStart = oldNum[0]
    oldEnd = oldNum[-1]
    lineStart = lineNum[0]
    lineEnd = lineNum[-1]
    return lineStart >= oldStart and lineEnd <= oldEnd
def updateMap(res, title, lineNum):
    lineNum = [lineNum] if type(lineNum) == int else lineNum
    lineOldNum = res[title]
    for oldNum in lineOldNum:
        #print(oldNum)
        if smallerRange(oldNum, lineNum):
            return
    res[title].append(lineNum)
    
def subDict(dict1, dict2):
    res = {}
    for item in dict1.keys():
        if item not in dict2.keys():
            res[item] = dict1[item]
        elif dict1[item] - dict2[item] > 0:
            res[item] = dict1[item] - dict2[item]
    return res
    
def getCount(mapRes):
    resCount = {}
    for item in mapRes:
        if len(item) > 0:
            for bugName in item.keys():
                count = len(item[bugName])
                if bugName in resCount.keys():
                    resCount[bugName] += count
                else:
                    resCount[bugName] = count
    return resCount

def get(cc, toolNames):
    mapRes = []
    for index,row in cc.iterrows():
        #同行算消除
        res = {}
        for toolName in toolNames:
            #toolRes = row[toolName]
            toolRes =  eval(row[toolName]) if type(row[toolName]) == str else row[toolName]
            for tRes in toolRes:
                lineNum = tRes['lineno']
                title = mapp[toolName.split('_')[0]][tRes['title'].strip()]#已经转换过
                if title in res.keys():
                    updateMap(res, title, lineNum)
                else:
                    res[title] = [[lineNum] if type(lineNum) == int else lineNum]#使用lineNum--list的len计算数量 [[],[]]
            #temp = [res]
            #print(getCount(temp))
        mapRes.append(res)
    resCount =  getCount(mapRes)
    return resCount  
    
#gas function
#将结果分割，需要不同的split字符串
def getGasResList(address, num):
    pat = open(address,'r').read()
    patList = re.split('\n[0-9]+ : ', pat)#
    ans = {}
    for sol in patList:
        if 'success' not in sol:
            continue
        sol = sol.split('\n')
        sol = sol[:-1]
        if sol[0] in ans.keys():
            ans[sol[0]] += patternDeal(sol, num)
        else:
            ans[sol[0]] = patternDeal(sol, num)
    return ans

#匹配gasPattern与克隆对的范围
def matchRange(item, contract_line):
    return int(item['rows'][0].strip()) >= contract_line[0] and int(item['rows'][1].strip()) <= contract_line[1]

#当克隆对合约在patList内，并且范围正确，旧获得
def compareRange(pat, contract_line, name):

    return  [
        item['rows'] for item in pat[name] if matchRange(item, contract_line)
    ] if name in pat.keys() else None

#将gas结果加到bb内
def GasAppendByRange(bb,pat,num):
    bb['pattern{}'.format(num)] = [ compareRange(pat, item[1]['contract_line'], item[1]['code_name'].split('.')[0]) for item in bb.iterrows() ]

def runGasPattern(patNum, rootAddress, bb):
    for num in patNum:
    #切割
        pat = getGasResList(os.path.join(rootAddress,'gasDetectResult{}.txt').format(num), num)
        GasAppendByRange(bb, pat, num)

def patternDeal(sol, num):
    if num == 6:
        return [ 
                {
                    'itemName' : item.split(':')[0],
                    'rows' : [ i.strip() for i in item.split(':')[1].split(',') ]
                      }  for item in sol[1:] if ':' in item and 'uint' in item.split(':')[0] and 'uint256' not in item.split(':')[0]
                ]
    return [ 
                {
                    'itemName' : item.split(':')[0],
                    'rows' : [ i.strip() for i in item.split(':')[1].split(',') ]
                      }  for item in sol[1:] if ':' in item 
                ]
    
head = [' access_control ',
 ' arithmetic ',
 ' denial_service ',
 ' reentrancy ',
 ' unchecked_low_calls ',
 ' bad_randomness ',
 ' front_running ',
 ' time_manipulation ',
 ' short_addresses ',
 ' Other ',
 ' Ignore ']
dic = dict(zip(head,[0]*10))
mapping = open('/home/yfliu/mapping.json').read()
mapp = json.loads(mapping)

slither_map = mapp['slither']
slither_key = slither_map.keys()
slither_type_map = dict(zip(slither_key,[0]*len(slither_key)))
u = open('/home/yfliu/paper/trash/temp.txt').read()
for i in u.splitlines():
    sp = i.split()
    slither_type_map[sp[1]] = sp[-2]
slither_type_map
slither_type_map['constant-function'] = 'Medium'
slither_bug_patch = ['constable-states','erc20-indexed','erc20-interface','external-function','incorrect-equality','naming-convention','shadowing-local','shadowing-state','shadowing-builtin','shadowing-abstract']  

def checkMatch(res, match):
    for idx,row in match.iterrows():
        sni = row['snippet']
        code = row['code'][0]
        data = res[res['sni_index'] == sni]['code_index']
        #print(data)
        if len(data) > 0 and list(data)[0] != code:
            print('error:{} {}'.format(sni,code))

def run_sTs(sniPath, codePath, matchPath, storeAddr, bugResAddr): 

    bugRes = pd.read_pickle(bugResAddr)
    snippet = pd.read_csv(sniPath)
    code = pd.read_csv(codePath)
    match = pd.read_pickle(matchPath)
    #match['snippet'] = match.index
    if 'snippet' not in match.columns:
        if 'index' in match.columns:
            match['snippet'] = match['index']
        else:
            match['snippet'] = match.index
    match['code'] = match['0']
    match = match[match['code'].apply(lambda x : len(x) > 0 )]

    sni_index = 'snippet'
    code_index = 'code'
    sni_index = ''
    snippet = snippet[['sol_name','code']]
    code_name = []
    contract_code = []
    sni_name = []
    sni_code = []
    dealed_code = []
    for index,i in match.iterrows():
        a = snippet.loc[int(i['snippet'])]
        b = code.loc[i['code'][0]]#AST
        code_name.append(b['code_name'])
        contract_code.append(b['code'])
        dealed_code.append(b['dealed_code'])
        sni_name.append(a['sol_name'])
        sni_code.append(a['code'])
    bb = pd.DataFrame({
        'sni_index':list(match['snippet']),
        'code_index':[int(item[0]) for item in list(match['code'])],#ast
        'code_name':code_name,
        'dealed_code':dealed_code,
        'sni_name':sni_name,
        'contract_code':contract_code,
        'sni_code':sni_code
    })
    #为结果添加.sol方便下一步处理 确定contract在完整合约的范围 把工具结果(未处理)贴上去
    bname = bb['code_name']
    bb['contract_line'] = get_contract_line(bb)
    bb['name'] = bb['code_name'].apply(lambda x : x.split('.')[0])
    bb = bb.merge(bugRes,on='name')
    bb['mythril'] = bb['mythril'].apply(lambda x : json.loads(x) if type(x) == str else None)
    bb['slither'] = bb['slither'].apply(lambda x : json.loads(x) if type(x) == str else None)
    bb['osiris'] = bb['osiris'].apply(lambda x : json.loads(x) if type(x) == str else None)
    bb['osiris_analysis'] = osiris_get(bb)
    bb['mythril_analysis'] = mythril_get(bb)
    bb['slither_analysis'] = slither_get(bb)

    #gas
    rootAddress = '/home/yfliu/paper_data/gasPattern/new'
    patNum = [1,6,2,3,4,7]
    runGasPattern(patNum, rootAddress, bb)
    count = 0
    for item in patNum:
        print(item,end=' ')
        num = len(bb[bb['pattern{}'.format(item)].apply(lambda x : x != None and len(x) > 0)])
        print(num)
        count += num
    print(count)
    
    slither_temp = list(bb['slither_analysis'])
    slither_info = []
    for item in slither_temp:
        del_list = []
        info_list = []
        for index,t_item in enumerate(item):
            title = t_item['title']
            level = slither_type_map[title]
            if level == 'Informational' or title in slither_bug_patch:
                del_list.append(index)
        del_list.reverse()
        for tempt in del_list:
            info_list.append(item[tempt])
            del item[tempt]
        slither_info.append(info_list)
    bb['slither_analysis_not_info'] = slither_temp
    #bb['slither_analysis_info'] = slither_info
    slither_temp = list(bb['mythril_analysis'])
    slither_info = []
    for item in slither_temp:
        del_list = []
        info_list = []
        for index,t_item in enumerate(item):
            #title = t_item['title']
            level = t_item['type']
            if level != 'Warning':
                del_list.append(index)
        del_list.reverse()
        for tempt in del_list:
            info_list.append(item[tempt])
            del item[tempt]
        slither_info.append(info_list)
    bb['mythril_analysis_not_info'] = slither_temp

    cc = bb[bb['dealed_code'].apply(lambda x : len(x) > 2 )]
    cc = cc.reset_index()
    checkMatch(cc, match)
    toolNames = ['mythril_analysis_not_info','osiris_analysis','slither_analysis_not_info']
    print('{} {}'.format('mythril', get(cc, ['mythril_analysis_not_info'])))
    print('{} {}'.format('osiris', subDict(get(cc, ['mythril_analysis_not_info','osiris_analysis']), get(cc, ['mythril_analysis_not_info']))))
    print('{} {}'.format('slither', subDict(get(cc, ['mythril_analysis_not_info','osiris_analysis','slither_analysis_not_info']), get(cc, ['mythril_analysis_not_info','osiris_analysis']))))
    print('total:',end = ' ')
    total = get(cc, ['mythril_analysis_not_info','osiris_analysis','slither_analysis_not_info'])
    count = 0
    for item in total:
        count += total[item]
    print(total)
    print(count)
    cc.to_pickle(storeAddr)
    return cc
    
sniPathF = "/home/yfliu/paper_data/conFunFormat/snippet_function.csv"
sniPathC = "/home/yfliu/paper_data/conFunFormat/snippet_contract.csv"

codePathF = "/home/yfliu/paper_data/conFunFormat/code_function.csv"
codePathC = "/home/yfliu/paper_data/conFunFormat/code_contract.csv"

In [57]:
#ast-bleu-0.7
matchPathF = '/home/yfliu/paper_data/matchRes/function_bleu_ast_all.pic'
matchPathC = '/home/yfliu/paper_data/matchRes/contract_bleu_ast_all.pic'
bugResAddr = '/home/yfliu/paper_data/completeRes/astBleu7.pic'
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_bleu7.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_bleu7.pic'
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 70
6 249
2 8
3 6
4 29
7 11
373
mythril {' arithmetic ': 122, ' reentrancy ': 16, ' Other ': 11, ' front_running ': 12, ' access_control ': 5}
osiris {' arithmetic ': 117, ' reentrancy ': 1, ' Ignore ': 28, ' time_manipulation ': 10, ' denial_service ': 1}
slither {' reentrancy ': 23, ' Other ': 10, ' unchecked_low_calls ': 22, ' time_manipulation ': 7, ' denial_service ': 17, ' access_control ': 16}
total: {' arithmetic ': 239, ' reentrancy ': 40, ' Other ': 21, ' unchecked_low_calls ': 22, ' front_running ': 12, ' Ignore ': 28, ' time_manipulation ': 17, ' denial_service ': 18, ' access_control ': 21}
418
1 2
6 69
2 1
3 1
4 11
7 2
86
mythril {' Other ': 7, ' front_running ': 7, ' arithmetic ': 40, ' reentrancy ': 4, ' access_control ': 3}
osiris {' arithmetic ': 37, ' denial_service ': 1, ' Ignore ': 4, ' time_manipulation ': 6}
slither {' unchecked_low_calls ': 6, ' Other ': 2, ' reentrancy ': 8, ' time_manipulation ': 3, ' access_control ': 4}
total: {' unchecked_low_calls ': 6, '

In [58]:
c

,index,sni_index,code_index,code_name,dealed_code,sni_name,contract_code,sni_code,contract_line,name,...,mythril_analysis,slither_analysis,pattern1,pattern6,pattern2,pattern3,pattern4,pattern7,slither_analysis_not_info,mythril_analysis_not_info
0,0,2,46,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,"['contract', 'multisig', 'event', 'Depositaddr...",7089-1.sol,contract multisig {\n\n\n\n \n\n\n\n \n\...,contract multisig { event Deposit(address from...,"[570, 610]",0xcf2b19384ea3f027f8620db3f797ae61bb475125,...,[],[],None,None,None,None,None,None,[],[]
1,1,8,37299,0x24afa69210c46d703c693d0ab4295df080bd1c9b.sol,"['contract', 'Token', 'function', 'transferFro...",37927-0.sol,contract Token {\n function transferFrom(addre...,contract MyToken {\n function transferF...,"[66, 68]",0x24afa69210c46d703c693d0ab4295df080bd1c9b,...,[],[],None,None,None,None,None,None,[],[]
2,2,6238,37299,0x24afa69210c46d703c693d0ab4295df080bd1c9b.sol,"['contract', 'Token', 'function', 'transferFro...",37858-0-0.sol,contract Token {\n function transferFrom(addre...,contract TheTokenContract {\n \n functio...,"[66, 68]",0x24afa69210c46d703c693d0ab4295df080bd1c9b,...,[],[],None,None,None,None,None,None,[],[]
3,3,31,716,0x439f4b2d7b7dfcde4d63efb075ebe250613bebd6.sol,"['contract', 'mortal', 'address', 'owner', 'fu...",7028-0.sol,contract mortal\n{\n address owner;\n\n ...,contract mortal { address owner; function mort...,"[2, 34]",0x439f4b2d7b7dfcde4d63efb075ebe250613bebd6,...,[],[],None,"[[13, 13], [21, 21], [26, 26], [31, 31]]",None,None,None,None,[],[]
4,4,405,716,0x439f4b2d7b7dfcde4d63efb075ebe250613bebd6.sol,"['contract', 'mortal', 'address', 'owner', 'fu...",43339-0.sol,contract mortal\n{\n address owner;\n\n ...,contract qualification {\n\naddress owner;\n\n...,"[2, 34]",0x439f4b2d7b7dfcde4d63efb075ebe250613bebd6,...,[],[],None,"[[13, 13], [21, 21], [26, 26], [31, 31]]",None,None,None,None,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,576,6987,56570,0x937a7486f5eb354c4a77882542333c4e8919203a.sol,"['contract', 'SendToMany', 'address', 'public'...",29386-0-0.sol,contract SendToMany\n{\n address[] public r...,contract SendToMany\n{\n address[] public r...,"[2, 19]",0x937a7486f5eb354c4a77882542333c4e8919203a,...,[],"[{'title': 'calls-loop', 'function': 'fallback...",None,[],None,None,None,None,"[{'title': 'calls-loop', 'function': 'fallback...",[]
577,577,7049,49766,0xcf3676f07dcb4028c90fbecdc4ded0abcda1ed01.sol,"['contract', 'SimpleToken', 'is', 'ERC20', 'st...",66012-0-0.sol,contract SimpleToken is ERC20 {\n\n\n\n strin...,contract SimpleToken is ERC20 {\n\n string pu...,"[596, 626]",0xcf3676f07dcb4028c90fbecdc4ded0abcda1ed01,...,[],[],None,"[[605, 605]]",None,None,None,None,[],[]
578,578,7076,65200,0xfbeeb0b43213a0abaa921f65873e09849ae5deec.sol,"['contract', 'Empty', 'uint', 'public', 'value...",26053-1-1.sol,contract Empty {\n uint public value = 5;\n},contract A {\n uint public value = 1;\n},"[3, 5]",0xfbeeb0b43213a0abaa921f65873e09849ae5deec,...,[],[],None,"[[5, 5]]",None,None,None,None,[],[]
579,579,7093,36745,0x1c83501478f1320977047008496dacbd60bb15ef.sol,"['contract', 'ERC223Receiver', 'function', 'to...",34224-0-0.sol,contract ERC223Receiver {\n function tokenFal...,contract ERC223Receiver\n{\n function token...,"[1258, 1260]",0x1c83501478f1320977047008496dacbd60bb15ef,...,[],[],[],"[[1260, 1260]]",None,None,[],None,[],[]


In [59]:
f

,index,sni_index,code_index,code_name,dealed_code,sni_name,contract_code,sni_code,contract_line,name,...,mythril_analysis,slither_analysis,pattern1,pattern6,pattern2,pattern3,pattern4,pattern7,slither_analysis_not_info,mythril_analysis_not_info
0,0,4,127374,0x862723e22dd5aa4c0875eb0c4ba656f40c750f02.sol,"['function', 'TCToken', 'uint256', 'initialSup...",15989-0.sol,function TCToken(\n\n uint256 initialSu...,function MyToken(\n uint256 initialSupp...,"[130, 298]",0x862723e22dd5aa4c0875eb0c4ba656f40c750f02,...,[],[],[],"[[137, 137]]",None,None,None,None,[],[]
1,1,21,10964,0x59524c5088fa3a7a1c441940061219c65b486474.sol,"['function', 'assertbool', 'assertion', 'inter...",47952-0.sol,function assert(bool assertion) internal {\n ...,function assert(bool assertion) internal {\n ...,"[199, 203]",0x59524c5088fa3a7a1c441940061219c65b486474,...,[],[],None,[],None,None,None,[],[],[]
2,2,815,10972,0x59524c5088fa3a7a1c441940061219c65b486474.sol,"['function', 'finishMinting', 'public', 'onlyO...",25069-0.sol,function finishMinting() public onlyOwner {\n ...,function finishMinting() public onlyOwner {\n ...,"[551, 558]",0x59524c5088fa3a7a1c441940061219c65b486474,...,[],[],None,"[[553, 553], [554, 554]]",None,None,None,[],[],[]
3,3,1795,10973,0x59524c5088fa3a7a1c441940061219c65b486474.sol,"['function', 'retrieveTokensaddress', 'token',...",28842-0.sol,function retrieveTokens(address _token) public...,function retrieveTokens(address _token) public...,"[564, 567]",0x59524c5088fa3a7a1c441940061219c65b486474,...,[],"[{'title': 'unused-return', 'function': 'finis...",None,[],None,None,None,[],"[{'title': 'unused-return', 'function': 'finis...",[]
4,4,1840,10969,0x59524c5088fa3a7a1c441940061219c65b486474.sol,"['function', 'createTokensaddress', 'recipient...",25077-0.sol,function createTokens(address recipient) publi...,function createTokens(address recipient) publi...,"[484, 490]",0x59524c5088fa3a7a1c441940061219c65b486474,...,[],[],None,"[[486, 486], [487, 487]]",None,None,None,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,351,6053,22271,0x258f55d71cd6e2cc73f962f24e4d915ecf2e869c.sol,"['function', 'doPauseuint', 'act', 'public', '...",64025-0-0.sol,function _doPause(uint act) public{\n\n ...,function _doPause(uint someUint) public{\n ...,"[564, 572]",0x258f55d71cd6e2cc73f962f24e4d915ecf2e869c,...,[],[],None,"[[565, 565]]",None,None,[],None,[],[]
352,352,6139,116507,0x551d599a55b9080601c8f911e2327403014ca1ed.sol,"['function', 'PixelsMain', 'public', 'payable'...",33463-0-0.sol,function PixelsMain() public payable {\n ...,function BetOnBitcoin() payable public{\n ...,"[227, 229]",0x551d599a55b9080601c8f911e2327403014ca1ed,...,[],[],None,[],None,None,None,None,[],[]
353,353,6159,1066,0x2071f61de93ca756f83264a7f4a661ca0383742e.sol,"['function', 'withdraw', 'uint', 'balance', 'm...",7204-0-7.sol,function withdraw(){\n uint balance = m...,function withdraw(){\n uint balance = mainD...,"[12, 17]",0x2071f61de93ca756f83264a7f4a661ca0383742e,...,[],[],None,"[[14, 14]]",None,None,"[[16, 16], [16, 16]]",None,[],[]
354,354,6182,71570,0xed6ac8de7c7ca7e3a22952e09c2a2a1232ddef9a.sol,"['function', 'MyToken', 'uint256', 'initialSup...",5674-0-1.sol,function MyToken(\n uint256 initial...,function MyAdvancedToken(\n uint256 initial...,"[46, 107]",0xed6ac8de7c7ca7e3a22952e09c2a2a1232ddef9a,...,[],[],None,None,None,None,None,None,[],[]


In [50]:
#ast
matchPathF = "/home/yfliu/paper_data/matchRes/match_function_v2.pic"
matchPathC = "/home/yfliu/paper_data/matchRes/match_contract_v2.pic"  
bugResAddr = '/home/yfliu/paper_data/completeRes/match_bug_res_v2.pic'
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_v2.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_v2.pic'
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 105
6 394
2 16
3 6
4 40
7 22
583
mythril {' arithmetic ': 149, ' reentrancy ': 53, ' Other ': 11, ' front_running ': 12, ' access_control ': 5}
osiris {' arithmetic ': 119, ' reentrancy ': 1, ' Ignore ': 28, ' time_manipulation ': 11, ' denial_service ': 1}
slither {' Other ': 52, ' reentrancy ': 52, ' unchecked_low_calls ': 53, ' time_manipulation ': 45, ' denial_service ': 23, ' access_control ': 17}
total: {' arithmetic ': 268, ' Other ': 63, ' reentrancy ': 106, ' unchecked_low_calls ': 53, ' time_manipulation ': 56, ' front_running ': 12, ' Ignore ': 28, ' denial_service ': 24, ' access_control ': 22}
632
1 2
6 99
2 1
3 1
4 11
7 2
116
mythril {' Other ': 7, ' front_running ': 13, ' arithmetic ': 47, ' reentrancy ': 6, ' access_control ': 3}
osiris {' arithmetic ': 45, ' reentrancy ': 2, ' denial_service ': 2, ' Ignore ': 10, ' time_manipulation ': 6}
slither {' unchecked_low_calls ': 7, ' Other ': 3, ' reentrancy ': 9, ' denial_service ': 1, ' Ignore ': 1, ' time_manipulation ':

In [51]:
#nicad2c
matchPathF = "/home/yfliu/paper_data/matchRes/nicadFunMatch.pic"
matchPathC = "/home/yfliu/paper_data/matchRes/nicadConMatch.pic" 
bugResAddr = '/home/yfliu/paper_data/completeRes/nicadBugRes.pic'
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_nicad.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_nicad.pic'
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 45
6 120
2 4
3 4
4 22
7 6
201
mythril {' arithmetic ': 70, ' reentrancy ': 11, ' Other ': 6, ' front_running ': 7, ' access_control ': 1}
osiris {' arithmetic ': 72, ' Ignore ': 20, ' time_manipulation ': 5, ' reentrancy ': 1, ' denial_service ': 1}
slither {' time_manipulation ': 6, ' reentrancy ': 16, ' unchecked_low_calls ': 13, ' denial_service ': 6, ' Other ': 4, ' access_control ': 6}
total: {' arithmetic ': 142, ' Ignore ': 20, ' time_manipulation ': 11, ' reentrancy ': 28, ' unchecked_low_calls ': 13, ' denial_service ': 7, ' Other ': 10, ' front_running ': 7, ' access_control ': 7}
245
1 0
6 8
2 0
3 0
4 1
7 0
9
mythril {' Other ': 2, ' front_running ': 2, ' reentrancy ': 1, ' arithmetic ': 3}
osiris {' Ignore ': 1, ' arithmetic ': 10, ' time_manipulation ': 2}
slither {' reentrancy ': 1, ' unchecked_low_calls ': 1}
total: {' Other ': 2, ' front_running ': 2, ' Ignore ': 1, ' arithmetic ': 13, ' reentrancy ': 2, ' time_manipulation ': 2, ' unchecked_low_calls ': 1}
23


In [52]:
#nicad32c
matchPathC = '/home/yfliu/paper_data/matchRes/nicad32CConMatch.pic'
matchPathF = '/home/yfliu/paper_data/matchRes/nicad32CFunMatch.pic'
bugResAddr = '/home/yfliu/paper_data/completeRes/nicad32CBugRes.pic'
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_nicad32C.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_nicad32C.pic'
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 73
6 211
2 10
3 6
4 36
7 16
352
mythril {' arithmetic ': 193, ' Other ': 13, ' front_running ': 13, ' reentrancy ': 36, ' access_control ': 2}
osiris {' arithmetic ': 160, ' Ignore ': 29, ' time_manipulation ': 16, ' denial_service ': 1}
slither {' Ignore ': 1, ' time_manipulation ': 7, ' reentrancy ': 30, ' unchecked_low_calls ': 23, ' Other ': 11, ' access_control ': 8, ' denial_service ': 8}
total: {' arithmetic ': 353, ' Ignore ': 30, ' time_manipulation ': 23, ' reentrancy ': 66, ' unchecked_low_calls ': 23, ' Other ': 24, ' access_control ': 10, ' denial_service ': 9, ' front_running ': 13}
551
1 0
6 15
2 0
3 0
4 2
7 0
17
mythril {' arithmetic ': 1}
osiris {' time_manipulation ': 5, ' Ignore ': 2, ' arithmetic ': 8}
slither {' reentrancy ': 5, ' unchecked_low_calls ': 4, ' Other ': 2}
total: {' time_manipulation ': 5, ' reentrancy ': 5, ' Ignore ': 2, ' unchecked_low_calls ': 4, ' Other ': 2, ' arithmetic ': 9}
27


In [53]:
#manual
matchPathC = '/home/yfliu/paper_data/matchRes/manualMatchCon.pic'
matchPathF = '/home/yfliu/paper_data/matchRes/manualMatchFun.pic'
bugResAddr = '/home/yfliu/paper_data/completeRes/manualBugRes.pic'
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_manual.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_manual.pic' 
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 85
6 268
2 11
3 5
4 28
7 12
409
mythril {' arithmetic ': 129, ' reentrancy ': 29, ' Other ': 17, ' front_running ': 12, ' access_control ': 4}
osiris {' arithmetic ': 120, ' reentrancy ': 1, ' Ignore ': 21, ' time_manipulation ': 10, ' denial_service ': 1}
slither {' denial_service ': 13, ' Other ': 21, ' reentrancy ': 30, ' unchecked_low_calls ': 18, ' time_manipulation ': 21, ' access_control ': 7}
total: {' arithmetic ': 249, ' denial_service ': 14, ' Other ': 38, ' reentrancy ': 60, ' unchecked_low_calls ': 18, ' time_manipulation ': 31, ' front_running ': 12, ' Ignore ': 21, ' access_control ': 11}
454
1 0
6 19
2 1
3 1
4 3
7 2
26
mythril {' Other ': 3, ' front_running ': 2, ' arithmetic ': 10, ' reentrancy ': 1}
osiris {' arithmetic ': 10, ' Ignore ': 2, ' time_manipulation ': 1}
slither {' reentrancy ': 2}
total: {' Other ': 3, ' front_running ': 2, ' arithmetic ': 20, ' Ignore ': 2, ' time_manipulation ': 1, ' reentrancy ': 3}
31


In [54]:
#nicad3-0.5
bugResAddr = '/home/yfliu/paper_data/completeRes/nicad3BugRes0.5.pic'
matchPathC = "/home/yfliu/paper_data/matchRes/nicad3ConMatch_0.5.pic"
matchPathF = "/home/yfliu/paper_data/matchRes/nicad3FunMatch_0.5.pic"
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_nicad3_0.5.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_nicad3_0.5.pic'
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 115
6 305
2 11
3 6
4 48
7 31
516
mythril {' arithmetic ': 219, ' Other ': 19, ' front_running ': 24, ' reentrancy ': 33, ' access_control ': 4}
osiris {' arithmetic ': 229, ' Ignore ': 39, ' time_manipulation ': 19, ' denial_service ': 2}
slither {' reentrancy ': 33, ' unchecked_low_calls ': 31, ' Other ': 22, ' time_manipulation ': 4, ' access_control ': 8, ' denial_service ': 15}
total: {' arithmetic ': 448, ' Ignore ': 39, ' reentrancy ': 66, ' unchecked_low_calls ': 31, ' Other ': 41, ' time_manipulation ': 23, ' access_control ': 12, ' denial_service ': 17, ' front_running ': 24}
701
1 0
6 20
2 1
3 0
4 4
7 0
25
mythril {' arithmetic ': 7, ' front_running ': 1}
osiris {' arithmetic ': 11, ' Ignore ': 4, ' time_manipulation ': 8}
slither {' reentrancy ': 9, ' Other ': 4, ' unchecked_low_calls ': 5, ' access_control ': 1}
total: {' arithmetic ': 18, ' Ignore ': 4, ' time_manipulation ': 8, ' reentrancy ': 9, ' Other ': 4, ' unchecked_low_calls ': 5, ' front_running ': 1, ' access_c

In [55]:
#nicad3-0.8
bugResAddr = '/home/yfliu/paper_data/completeRes/nicad3BugRes0.8.pic'
matchPathC = "/home/yfliu/paper_data/matchRes/nicad3ConMatch_0.8.pic"
matchPathF = "/home/yfliu/paper_data/matchRes/nicad3FunMatch_0.8.pic"
funStoreAddr = '/home/yfliu/paper_data/astResultCsv/functionRes_nicad3_0.8.pic'
conStoreAddr = '/home/yfliu/paper_data/astResultCsv/contractRes_nicad3_0.8.pic'
c = run_sTs(sniPathC, codePathC, matchPathC, conStoreAddr, bugResAddr)
f = run_sTs(sniPathF, codePathF, matchPathF, funStoreAddr, bugResAddr)

1 367
6 1350
2 21
3 32
4 170
7 91
2031
mythril {' arithmetic ': 1158, ' front_running ': 77, ' Other ': 65, ' reentrancy ': 153, ' access_control ': 71}
osiris {' arithmetic ': 658, ' Ignore ': 195, ' time_manipulation ': 59, ' reentrancy ': 29, ' denial_service ': 43}
slither {' reentrancy ': 58, ' unchecked_low_calls ': 107, ' Other ': 102, ' time_manipulation ': 85, ' access_control ': 99, ' denial_service ': 102}
total: {' arithmetic ': 1816, ' Ignore ': 195, ' reentrancy ': 240, ' unchecked_low_calls ': 107, ' front_running ': 77, ' Other ': 167, ' time_manipulation ': 144, ' access_control ': 170, ' denial_service ': 145}
3061
1 0
6 57
2 1
3 4
4 9
7 3
74
mythril {' arithmetic ': 11, ' front_running ': 2}
osiris {' arithmetic ': 32, ' Ignore ': 6, ' time_manipulation ': 20}
slither {' reentrancy ': 14, ' time_manipulation ': 2, ' access_control ': 4, ' unchecked_low_calls ': 8, ' Other ': 11, ' denial_service ': 2}
total: {' arithmetic ': 43, ' Ignore ': 6, ' reentrancy ': 14, ' t